# Rules

**Dates**: 23.12.2024 - 29.12.2024

**Deadline**: 29.12.2024 23:59

**Submission after deadline**: NOT POSSIBLE


For each task you'll get a certain ammount of points (indicated in brackets next to the task). The maximum grade for the work is 10 points.

* **Tasks, where you should write code**, are marked with 🐍 (snake emoji).

* **Questions, where you should write your comments**, are marked with ❓ (question emoji).

You'll also see "assert" code, this code is ought to help you: if you're doing everything right - the code won't show AssertionError error.

**Disclaimer**: the exam must be completed independently. "Very similar" solutions are considered plagiarism and all students involved (including those from whom the work was copied) cannot receive more than 0 points for it.


# Preparations

Note, that you should import additional libraries and modules for the solution.

In [ ]:
# Handling data
import pandas as pd
import numpy as np

# Visualizations
import matplotlib.pyplot as plt
import plotly.express as px

# Tests
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller, kpss

# Decomposition
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL as STL_decomp
from statsmodels.tsa.seasonal import MSTL as MSTL_decomp

# Prophet model
import prophet as fp

# Tuning
import itertools

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install pmdarima

# auto ARIMA
from pmdarima import auto_arima

In [ ]:
!pip install statsforecast

# MSTL
from statsforecast import StatsForecast
from statsforecast.models import (
    MSTL
  )

In [ ]:
def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

In [ ]:
def kpss_test(timeseries):
    print("Results of KPSS Test:")
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    print(kpss_output)

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())


# Problem definition

You will work on forecasting metrics of *BookBnb*, a short-term rental platform, which is loved by travelers by its services and quality of properties. You will forecast metric of **views** for listings (=properties) from *Kaliningrad*, listed withing the platform.

The **views** metric represents the number of times a property listing is viewed by users on the platform. It serves as an indicator of interest or demand for a particular property or destination. This metric is crucial for understanding user engagement, the attractiveness of listings and overall platform activity. Thus, **views** in Kaliningrad reflect the level of interest in this region, making it a key metric for evaluating its popularity and potential for further investment.

The primary objective is to evaluate the popularity of Kaliningrad as a destination on the platform and determine whether it's worth investing in advertising properties in this region. Your forecast will guide BookBnb in making strategic decisions about resource allocation and marketing campaigns to drive growth and enhance user experience.

Your goal is to build a reliable forecast for the next year and prove that the company can trust it. This forecast will serve as a foundation for making informed decisions in the competitive rental market. Good luck! 🚀

Note the forecast horizon: your final task is to forecast one future year.

In [ ]:
FORECAST_HORIZON = 365

# Data

You can download the dataset *exam_views_kgd.csv* with time series for *views* from the [link](https://drive.google.com/file/d/1XRh0JKvFGtNjOJXG4_qZBDWX9R5FXldb/view?usp=sharing).

The dataset includes historical data **from the start of 2019 to the end of 2022** (imagine it's 2022 now :D).

**Columns:**

* *region* - region, where properties are present, in this dataset only "Kaliningrad" is present;

* *dt* - dates, daily, no absent dates;

* *metric* - name of the metric, in this dataset only "views" is present;

* *value* - value of the metric.

In [ ]:
df = pd.read_csv('exam_views_kgd.csv', sep=';', parse_dates=['dt'])
df

,region,dt,metric,value
0,Kaliningrad,2019-01-01,views,26011.0
1,Kaliningrad,2019-01-02,views,23158.0
2,Kaliningrad,2019-01-03,views,23413.0
3,Kaliningrad,2019-01-04,views,22534.0
4,Kaliningrad,2019-01-05,views,21431.0
...,...,...,...,...
1456,Kaliningrad,2022-12-27,views,23796.0
1457,Kaliningrad,2022-12-28,views,22762.0
1458,Kaliningrad,2022-12-29,views,20338.0
1459,Kaliningrad,2022-12-30,views,10723.0


Let's plot the time series.

In [ ]:
fig = px.line(title="BookBnb views for listings from Kaliningrad, daily")
fig.add_scatter(x=df.dt, y=df['value'], mode='lines')

fig.update_layout(template='plotly_white', width=1000, height=500)
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="views")
fig.show()

For further work columns should be renamed.

In [ ]:
df.columns = ['region', 'ds', 'metric', 'y']

df

,region,ds,metric,y
0,Kaliningrad,2019-01-01,views,26011.0
1,Kaliningrad,2019-01-02,views,23158.0
2,Kaliningrad,2019-01-03,views,23413.0
3,Kaliningrad,2019-01-04,views,22534.0
4,Kaliningrad,2019-01-05,views,21431.0
...,...,...,...,...
1456,Kaliningrad,2022-12-27,views,23796.0
1457,Kaliningrad,2022-12-28,views,22762.0
1458,Kaliningrad,2022-12-29,views,20338.0
1459,Kaliningrad,2022-12-30,views,10723.0


# 1 - Preprocessing

We should always start with preprocessing. The real-world data may contain nans, anomalies and other problems.

For this task, we'll consider nans.

## Task 1.1 - Filling nans (0.5 points)

🐍 **Detect nans in your dataset and choose proper method to fill nans.**

In [ ]:
# your code here
# ≽^•⩊•^≼

Plot data with filled nans.

In [ ]:
fig = px.line(title="BookBnb views for listings from Kaliningrad, daily")
fig.add_scatter(x=df.ds, y=df['y'], mode='lines')

fig.update_layout(template='plotly_white', width=1000, height=500)
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="views")
fig.show()

❓ **Why have you chosen this method to fill nans? Write your comments below:**

your answer here



In [ ]:
# Check if everything is ok

assert df.isna().sum().sum() == 0

# 2 - Regressors

To enhance the accuracy of your forecast, you'll incorporate regressors, both dummy and continuous. By leveraging these factors, you can better account for external events and trends that drive user behavior on the platform.

For this task, you'll consider the following regressors:

* **COVID-19**: the period of the pandemic, should have impacted travel behavior and interest in short-term rentals.

* **New product feature introduction**: a new search algorithm has been introduced to the platform at the beginning of 2022, that may have influenced user engagement.

* **DAU** (Daily Active Users) of the platform: the total number of active users on the platform each day, which provides a sense of overall user activity and engagement.

These regressors will help you capture the broader context and underlying factors influencing views, making your forecast more robust and insightful.

As a result of this part, you'll construct 2 dataframes:

*   **regressors** - dataset with your regressors on the period of your dataset (from 2019 to the end of 2022);
*   **regressors_future** - dataset with your regressors for the future dates (future *FORECAST_HORIZON* days).

In [ ]:
# Create regressors dataframe

regressors = pd.DataFrame()
regressors['ds'] = df.ds
regressors

,ds
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-05
...,...
1456,2022-12-27
1457,2022-12-28
1458,2022-12-29
1459,2022-12-30


In [ ]:
# Create regressors_future dataframe

regressors_future = pd.DataFrame()
regressors_future['ds'] = pd.date_range(start=df.ds.max()+ np.timedelta64(1, 'D'), periods=FORECAST_HORIZON)
regressors_future

,ds
0,2023-01-01
1,2023-01-02
2,2023-01-03
3,2023-01-04
4,2023-01-05
...,...
360,2023-12-27
361,2023-12-28
362,2023-12-29
363,2023-12-30


## 2.1 - Dummy regressors

It's time to prepare COVID-19 and new feature dummy regressors.

### Task 2.1 - Dummy regressors preparation (1 point)

1) Let's assume these dates for COVID-19: from 30 January 2020 up to 5 May 2023.

🐍 **Prepare a dummy regressor COVID-19 and add it to *regressors* and *regressors_future* dataframes.**

In [ ]:
# your code here
# ≽^•⩊•^≼

regressors['covid'] = # your code here
regressors_future['covid'] = # your code here

2) A new search engine was introduced on 1 January 2022.

🐍 **Prepare a dummy regressor for new feature and add it to *regressors* and *regressors_future* dataframes.**

In [ ]:
# your code here
# ≽^•⩊•^≼

regressors['new_feature'] = # your code here
regressors_future['new_feature'] = # your code here

## 2.2 - DAU continuous regressor

*DAU* should be a strong regressor for predicting *views*, because it reflects the overall activity level on the platform. An increase in *DAU* often indicates more users actively engaging with the app, leading to higher search activity and more property views. This connection makes DAU a good proxy for demand and user interest, capturing behavioral trends like spikes during holidays or after marketing campaigns.

You can download the dataset *exam_dau_total.csv* with time series for *DAU* from the [link](https://drive.google.com/file/d/11ScbYHtuXlG4siBDw5p9aDN47l6OrMH3/view?usp=sharing).

The dataset includes historical data **from the start of 2019 to the end of 2022**. Note that this is DAU for the whole platform.

**Columns:**


* *dt* - dates, daily, no absent dates;

* *metric* - name of the metric, in this dataset only "DAU" is present;

* *value* - value of the metric.

In [ ]:
reg = pd.read_csv('exam_dau_total.csv', sep=';', parse_dates=['dt'])
reg

,dt,metric,value
0,2019-01-01,DAU,1541086.0
1,2019-01-02,DAU,1766712.0
2,2019-01-03,DAU,1800857.0
3,2019-01-04,DAU,1832445.0
4,2019-01-05,DAU,1871104.0
...,...,...,...
1456,2022-12-27,DAU,2357898.0
1457,2022-12-28,DAU,2305066.0
1458,2022-12-29,DAU,2293198.0
1459,2022-12-30,DAU,2243752.0


Let's plot the regressor.

In [ ]:
fig = px.line(title="BookBnb DAU, daily")
fig.add_scatter(x=reg.dt, y=reg['value'], mode='lines')

fig.update_layout(template='plotly_white', width=1000, height=500)
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="DAU")
fig.show()

Don't forget, that usually you'll also have to preprocess your regressors.

🐍 **Check regressors for nulls and use a proper method to fill them.**

In [ ]:
# your code here
# ≽^•⩊•^≼

Plot *DAU* regressor with filled nans.

In [ ]:
fig = px.line(title="BookBnb DAU, daily")
fig.add_scatter(x=reg.dt, y=reg['value'], mode='lines')

fig.update_layout(template='plotly_white', width=1000, height=500)
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="DAU")
fig.show()

The main problem with this regressor is that it reflects *DAU* for the whole *BookBnb* platform and is quite noisy. Thus, let's not include the whole *DAU* time series, but extract the trend and use it as a regressor.

### Task 2.2.1 - Trend extraction (0.5 point)

Time series components can be extracted with proper decomposition algorythm.

🐍 **Extract trend with proper decomposition method.**

In [ ]:
# your code here
# ≽^•⩊•^≼

model = # your code here
res = model.fit()

fig = res.plot()
fig.set_size_inches(10, 6)
plt.show()

❓ **Why have you chosen this decomposition method and such lengths for seasonal period/periods? Write your comments below:**

your answer here

🐍 **Analyze residuals for autotocorrelation (with 1 test) and stationarity (with 2 tests).**

In [ ]:
# your code here
# ≽^•⩊•^≼

❓ **Is your decomposition successful based on the residuals? What may be the reasons for such a result? Write your comments below:**

your answer here

🐍 **Save the trend component from your decomposition to trend_reg variable.**

In [ ]:
trend_reg = # your code here

With the code below, you can see how *DAU trend* regressor "correlates" with *views* time series.

In [ ]:
import plotly.graph_objects as go

# Create an empty fig
fig = go.Figure()

# Add y graph on the left
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['y'],
    mode='lines',
    name='y',
    line=dict(color='blue'),
    yaxis='y1'  # Tie to y1
))

# Add x graph on the right
fig.add_trace(go.Scatter(
    x=trend_reg.index,
    y=trend_reg,
    mode='lines',
    name='net income',
    line=dict(color='red'),
    yaxis='y2'  # Tie to y1
))


fig.update_layout(
    title="BookBnb views for listings from Kaliningrad, daily",
    template='plotly_white',
    width=1000,
    height=500,
    yaxis=dict(
        title='stock prices',
    ),
    yaxis2=dict(
        title='net income',
        overlaying='y',  # Impose axes
        side='right'
    ),
    xaxis=dict(title='Date')
)

fig.show()

### Task 2.2.2 - Trend forecasting with Auto-ARIMAX (1 point)

After isolating the trend component of DAU, the next step is to model and forecast this trend for future dates.

Since trends often exhibit autocorrelation and gradual shifts over time, a statistical model like Auto-ARIMA is well-suited for this task.

As far as most businesse have been influenced by COVID-19, you should also add your COVID dummy regressor and build Auto-ARIMAX model.

🐍 **Build Auto-ARIMAX model (from ```pmdarima```), which takes COVID regressor into consideration.**

In [ ]:
# your code here
# ≽^•⩊•^≼

model = # your code here

❓ **How does auto-ARIMA choose proper orders (p,d,q)? Write your comments below:**

your answer here

Let's see the summary of the model.

In [ ]:
print(model.summary())

❓ **Write the equation for the built Auto-ARIMAX model based on the summary:**

your answer here


🐍 **Make out-of-sample forecast for future FORECAST_HORIZON dates.**

In [ ]:
# your code here
# ≽^•⩊•^≼

trend_forecast = # your code here

Let's plot the trend and its Auto-ARIMA forecast.

In [ ]:
fig = px.line(title="BookBnb DAU trend, daily")
fig.add_scatter(x=trend_reg.index, y=trend_reg, mode='lines', name='DAU', line=dict(color='blue'))
fig.add_scatter(x=trend_forecast.index, y=trend_forecast, mode='lines', name='DAU forecast', line=dict(color='red'))

fig.update_layout(template='plotly_white', width=800, height=500)
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="DAU")
fig.show()

🐍 **Add *trend_reg* to *regressors* dataframe and *trend_forecast* to *regressors_future* dataframe.**

In [ ]:
# your code here
# ≽^•⩊•^≼

regressors['DAU'] = # your code here
regressors_future['DAU'] = # your code here

In [ ]:
# Check if everything is ok

assert len(regressors.columns) == 4
assert len(regressors_future.columns) == 4

assert regressors.isna().sum().sum() == 0
assert regressors_future.isna().sum().sum() == 0

## Prepare train-test

Now we can move to preparing our dataset for the forecasting models.

🐍 **Add prepared regressors to initial time series.** Forecasting models, you'll be working today with, expect to have regressors in columns.

In [ ]:
# your code here
# ≽^•⩊•^≼

df_with_reg = # your code here

Let's split *views* time series into train/test: 2019-2021 for train and 2022 - for test

🐍 **Split target time series into train/test: 2019-2021 for train and 2022 - for test. Define the forecasting horizon.**

In [ ]:
# your code here
# ≽^•⩊•^≼

train_size = # your code here
train, test = df_with_reg[:train_size], df_with_reg[train_size:]

In [ ]:
# your code here
# ≽^•⩊•^≼

forecast_horizon = # your code here
forecast_horizon

In [ ]:
# Check if everything is ok

assert train.shape == (1096, 7)
assert test.shape == (365, 7)

Let's plot train/test split.

In [ ]:
fig = px.line(title="train-test")
fig.add_scatter(x=train['ds'], y=train['y'], mode='lines', name='train', line=dict(color='blue'))
fig.add_scatter(x=test['ds'], y=test['y'], mode='lines', name='test', line=dict(color='green'))

fig.update_layout(template='plotly_white', width=1000, height=500)
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="closing price")
fig.show()

# 3 - Prophet model

Finally, we can move to the models. To forecast *views* time series, we will use the Prophet model. Prophet is a ML forecasting tool designed to work effectively with time series data that exhibits seasonality, trends and holiday effects.

However, before you start you should choose a proper quality metric.

## Task 3.1 - Quality metric choice (0.5 points)

❓ **What quality metric have you chosen for the task? Why? Write your comments below:**

your answer here

## Task 3.2 - Prophet model

Further, you should add holidays and regressors to your model. Prophet treats both of these components separately and adding them might enhance your forecast.

### Task 3.2.1 - Holidays & Regressors (0.5 point)

🐍 **Prepare *holidays_df* dataframe to account for holidays in your Prophet model.**

In [ ]:
# Here's a dict of Russian holidays you should take into account

russian_holidays = {
    "New Year": "01-01",
    "Christmas": "01-07",
    "Defender of the Fatherland Day": "02-23",
    "International Women's Day": "03-08",
    "Spring and Labour Day": "05-01",
    "Victory Day": "05-09",
    "Day of Russia": "06-12",
    "National Unity Day": "11-04"
}

In [ ]:
dates = []
holidays = []

for year in pd.date_range(start=df.ds.min(), end=df.ds.max(), freq='YS'):
  for holiday in russian_holidays.keys():
    dates.append(str(year.year)+'-'+russian_holidays[holiday])
    holidays.append(holiday)

In [ ]:
# your code here
# ≽^•⩊•^≼

holidays_df = # your code here

Usually, the New Year's effect is present 10 days before and after 1 January.

🐍 **Add lower and upper windows for New Year holiday.**

In [ ]:
# your code here
# ≽^•⩊•^≼

In [ ]:
# Check if everything is ok
assert holidays_df.shape == (32, 4)

### Task 3.2.2 - Prophet model (0.5 point)

🐍 **Build, train Prophet model (from ```prophet```) with additive weekly seasonality to handle consistent weekly changes and multiplicative yearly seasonality. Add holidays and all of the regressors, considered in part 2. Build forecast.**

In [ ]:
# your code here
# ≽^•⩊•^≼

model_prophet = # your code here

# your code here

In [ ]:
# Build forecast
forecast_prophet = # your code here

Plot the forecast.

In [ ]:
fig = px.line(title="BookBnb views for listings from Kaliningrad forecast, daily")
fig.add_scatter(x=train['ds'], y=train['y'], mode='lines', name='train', line=dict(color='blue'))
fig.add_scatter(x=test['ds'], y=test['y'], mode='lines', name='test', line=dict(color='green'))
fig.add_scatter(x=here['ds'], y=here['yhat'], mode='lines', name='forecast', line=dict(color='red'))


fig.update_layout(template='plotly_white', width=1000, height=500)
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="views")
fig.show()

🐍 **Evaluate your forecast with chosen quality metric.**

In [ ]:
# your code here
# ≽^•⩊•^≼

Further, use the model with regressors and holidays.

## Task 3.3 - Hyperparameters tuning (1 point)

As a ML forecasting model, Prophet requires accurate hyperparameters tuning. Further, you'll tune hyperparameters with GridSearch + Cross Validation approach.



🐍 **Prepare hyperparameters grid.** Your grid should include parameters for trend, seasonality and holidays.

In [ ]:
# your code here
# ≽^•⩊•^≼

param_grid = {
    # your code for the grid here
}

# Create all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

len(all_params)

🐍 **Perform GridSearch + Cross Validation.**

In [ ]:
# your code here
# ≽^•⩊•^≼

🐍 **Extract hyperparameters which provide the best quality.**

In [ ]:
# your code here
# ≽^•⩊•^≼

best_params = # your code here
print(best_params)

🐍 **Build, train and forecast with tuned Prophet model.**

In [ ]:
# your code here
# ≽^•⩊•^≼

model_prophet_tuned = # your code here

# your code here

In [ ]:
# Build forecast
forecast_prophet_tuned = # your code here

Plot forecast.

In [ ]:
fig = px.line(title="BookBnb views for listings from Kaliningrad forecast, daily")
fig.add_scatter(x=train['ds'], y=train['y'], mode='lines', name='train', line=dict(color='blue'))
fig.add_scatter(x=test['ds'], y=test['y'], mode='lines', name='test', line=dict(color='green'))
fig.add_scatter(x=forecast_prophet_tuned['ds'], y=forecast_prophet_tuned['yhat'], mode='lines', name='forecast', line=dict(color='violet'))


fig.update_layout(template='plotly_white', width=1000, height=500)
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="views")
fig.show()

🐍 **Evaluate your forecast with chosen quality metric.**

In [ ]:
# your code here
# ≽^•⩊•^≼

# 4 - MSTL model

Now let's move to the MSTL (Multiplicative Seasonal-Trend Decomposition using LOESS), a statistical forecasting model. In this part, you will use MSTL as a forecasting tool.

## Task 4.1 - MSTL model (1 point)

🐍 **Build, train and forecast with MSTL model (from ```statsforecast```)  with regressors, discussed in part 2.**

In [ ]:
# your code here
# ≽^•⩊•^≼

❓ **Why have you chosen such lengths for seasonal period/periods? Write your comments below:**

your answer here

In [ ]:
# Build forecast
forecast_mstl = # your code here

In [ ]:
fig = px.line(title="BookBnb views for listings from Kaliningrad forecast, daily")
fig.add_scatter(x=train['ds'], y=(train['y']), mode='lines', name='train', line=dict(color='blue'))
fig.add_scatter(x=test['ds'], y=test['y'], mode='lines', name='test', line=dict(color='green'))
fig.add_scatter(x=forecast_mstl['ds'], y=(forecast_mstl['MSTL']), mode='lines', name='forecast', line=dict(color='red'))

fig.update_layout(template='plotly_white', width=1000, height=500)
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="views")
fig.show()

🐍 **Evaluate your forecast with chosen quality metric.**

In [ ]:
# your code here
# ≽^•⩊•^≼

# 5 - Comparison of models

As you've built 2 different models, it's now time to choose the best one. For this, you should perform Cross Validation (CV) to evaluate your model properly. Note, that if you use different libraries for Cross Validation, it's vital to obtain similar cutoffs for your models.

## Task 5.1 - Evaluate your best Prophet model with CV (1 point)

In [ ]:
# your code here
# ≽^•⩊•^≼

df_cv_prophet = # your code here to prepare a dataset with all the cutoffs of CV

In [ ]:
cv_metrics_prophet = # your code here

## Task 5.3 - Evaluate your best MSTL models with CV  (1 point)

In [ ]:
# your code here
# ≽^•⩊•^≼

df_cv_mstl = # your code here to prepare a dataset with all the cutoffs of CV

In [ ]:
cv_metrics_mstl = # your code here

In [ ]:
# Check if everything is ok

assert list(df_cv_prophet.cutoff.unique()) == list(df_cv_mstl.cutoff.unique())

## Task 5.4 - The best model (0.5 points)

❓ **Why have you chosen such parameters for your Cross Validations? Write your comments below:**

your answer here

❓ **Which model is the best? How have you chosen the best model? Why this particular model might have been chosen as the best one? Write your comments below:**

your answer here

# 6 - Forecasts

Finally, after the model has been tested and proved to be great,we're ready to provide forecast for the future year.

## Task 6.1 - Out-of-sample forecasts with the best model (1 point)

🐍 **Train your best forecasting model on the full time series. Build out-of-sample forecast for 1 future year.**

In [ ]:
# your code here
# ≽^•⩊•^≼

best_model = # your code here

# your code here

In [ ]:
# Build forecast
forecast_future = # your code here

Plot forecast.

In [ ]:
fig = px.line(title="BookBnb views for listings from Kaliningrad forecast, daily")
fig.add_scatter(x=train['ds'], y=train['y'], mode='lines', name='train', line=dict(color='blue'))
fig.add_scatter(x=test['ds'], y=test['y'], mode='lines', name='test', line=dict(color='green'))
fig.add_scatter(x=forecast_future['ds'], y=forecast_future['yhat'], mode='lines', name='forecast', line=dict(color='violet'))


fig.update_layout(template='plotly_white', width=1000, height=500)
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="views")
fig.show()

**That's the end, congratulations** ٩(⁎❛ᴗ❛⁎)۶